<a href="https://colab.research.google.com/github/s1scottd/CIFAR-10_Image-Classification-CNN/blob/main/CIFAR_10_Image_Classification_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIFAR-10 Image Classification with CNN

Convolutional Neural Network (CNN) model for image classification on the CIFAR-10 dataset.

## Table of Contents

1. [Introduction](#introduction)
2. [Technologies](#technologies)
3. [Installation](#installation)
4. [Data](#data)
5. [Methodology](#methodology)
6. [Results](#results)
7. [Conclusion](#conclusion)
8. [References](#references)

## Introduction <a name="introduction"></a>

This project involves building and training a Convolutional Neural Network to classify images from the CIFAR-10 dataset.  The project is a multiclass, single label problem.

## Technologies <a name="technologies"></a>

The project uses the following technologies and libraries:

- Python: 3.10.11
- TensorFlow: 2.12.1
- NumPy: 1.22.4
- Keras: 2.12.1

## Installation <a name="installation"></a>

Follow these steps to set up the project:

1. Clone the repository: `git clone https://github.com/s1scottd/CIFAR-10_classification-CNN.git`
2. Open Google Colab and upload the notebook file `CIFAR-10_classification-CNN.ipynb` or select the "Open in Colab" button at the top of jupyter file.

## Data <a name="data"></a>

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 40000 training images, 10000 validation images and 10000 test images.

## Baseline <a name="baseline"><a>
The baseline is a validation accuracy greater than 0.1. (random classifier)

## Measure of Success
This is a balanced classification problem, so the measure of success will be accuracy.

## Methodology <a name="methodology"></a>

The project involves the following steps:

1. Load and preprocess the data: The CIFAR-10 data is loaded using Keras and normalized to have pixel values between 0 and 1.
2. Define the CNN model: A CNN model is defined using Keras with two convolutional layers, a max pooling layer, and two dense layers.
3. Compile and train the model: The model is compiled and trained using the Adam optimizer and categorical cross-entropy loss function for 20 epochs.
4. Evaluate the model: The model's performance is evaluated on the test data using accuracy as the metric.

## Results <a name="results"></a>

The trained model achieves an accuracy of xx% on the test data. For a more detailed view of the model's performance and visualizations, check the Jupyter notebook `CIFAR-10_Classification.ipynb`.

## Conclusion <a name="conclusion"></a>

This project demonstrates the effectiveness of Convolutional Neural Networks in image classification tasks. Future work might involve exploring different model architectures, or using data augmentation techniques.

## References <a name="references"></a>

- O'Malley, Tom and Bursztein, Elie and Long, James and Chollet, Fran\c{c}ois and Jin, Haifeng and Invernizzi, Luca and others. KerasTuner, github.com/keras-team/keras-tuner. 
- Chollet, F. et al., 2015. Keras.

## Import Libraries. 

In [ ]:
import os
import tensorflow as tf
import seaborn as sns
import matplotlib as mplt
import matplotlib.pyplot as plt
import matplotlib.gridspec as gs
import numpy as np

from tensorflow import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

import pandas as pd

from platform import python_version
print(f"python version: {python_version()}")
print(f"tensorflow version: {tf.__version__}")
print(f"numpy version: {np.__version__}")
print(f"keras version: {keras.__version__}")


##View the Raw Data##

In [ ]:
# Load CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

print(f"X_train shape: {X_train.shape} and y_train shape: {y_train.shape}\n")
print(f"X_test shape:  {X_test.shape} and y_test shape: {y_test.shape}\n")
print(f"First raw value of X_train:\n {X_train[0]}\n")
print(f"First raw value of y_train:\n {y_train[0]}\n")
print(f"First raw value of X_test:\n {X_test[0]}\n")
print(f"First raw value of y_test:\n {y_test[0]}\n")

Check for missing data in the CIFAR-10 Dataset

In [ ]:
print(f"X_train contains NaN: {np.any(np.isnan(X_train))}")
print(f"X_test contains NaN: {np.any(np.isnan(X_test))}")
print(f"y_train contains NaN: {np.any(np.isnan(y_train))}")
print(f"y_test contains NaN: {np.any(np.isnan(y_test))}\n")

Split X_train into a training dataset and a validation dataset.

In [ ]:
X_train = X_train[:-10000]
X_val = X_train[-10000:]
y_train = y_train[:-10000]
y_val = y_train[-10000:]

Distribution of the data

In [ ]:
# Convert y_train, y_val and y_test into DataFrames
y_train_df = pd.DataFrame(y_train, columns=['Classes'])
y_val_df = pd.DataFrame(y_val, columns=['Classes'])
y_test_df = pd.DataFrame(y_test, columns=['Classes'])

fig, axs = plt.subplots(1,3,figsize=(15,5))

# Count plot for training set
sns.countplot(data=y_train_df, x='Classes', ax=axs[0])
axs[0].set_title('Distribution of training data')

# Count plot for training set
sns.countplot(data=y_val_df, x='Classes', ax=axs[1])
axs[1].set_title('Distribution of validation data')

# Count plot for testing set
sns.countplot(data=y_test_df, x='Classes', ax=axs[2])
axs[2].set_title('Distribution of Testing data')

plt.show()

Define an image display function

In [ ]:


def display_images(data, labels):
  
  labels = labels.reshape(-1)

  labels_dict = {
    0: 'airplane',
    1: 'automobile',
    2: 'bird',
    3: 'cat',
    4: 'deer',
    5: 'dog',
    6: 'frog',
    7: 'horse',
    8: 'ship',
    9: 'truck'
  }

  fig, axes = plt.subplots(3, 3, figsize=(10, 10))

  for i, ax in enumerate(axes.flat):
    img = data[i]
    label = labels[i]
    
    ax.imshow(img)
    ax.set_title(f"{labels_dict[label]}")
    ax.axis('off')

  plt.show()

Display the first 9 images

In [ ]:
# visualize data by plotting images
display_images(X_train, y_train)

##Prepare the data

Define a normalize function

In [ ]:
def normalize(data):
  data = data.astype("float32")/255
  return data

Normalize the dataset

In [ ]:
X_train = normalize(X_train)
X_test = normalize(X_test)
X_val = normalize(X_val)

print(X_train.shape[0], 'train samples')
print(X_val.shape[0], 'validation samples')
print(X_test.shape[0], 'test samples')

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape:  {X_test.shape}")
print(f"X_val shape: {X_val.shape}")


Convert class vectors to binary class matrices - one hot encoding.

In [ ]:
# Convert class vectors to binary class matrices. This is called one hot encoding.
y_train = keras.utils.to_categorical(y_train, 10)
y_val = keras.utils.to_categorical(y_val, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# Convolutional Neural Network without Tuning

##Baseline Model##

Define a baseline model and compile 

In [ ]:
def build_baseline_model():
  model = Sequential()
# CONV => RELU => CONV => RELU => POOL => DROPOUT
  model.add(Conv2D(32, (3, 3), padding='same',input_shape=X_train.shape[1:]))
  model.add(Activation('relu'))
  model.add(Conv2D(32, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

# CONV => RELU => CONV => RELU => POOL => DROPOUT
  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(64, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

# FLATTERN => DENSE => RELU => DROPOUT
  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
# a softmax classifier
  model.add(Dense(10))
  model.add(Activation('softmax'))

    # Compile the model
  model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss=tf.keras.losses.CategoricalCrossentropy(), 
              metrics=['accuracy'])

  return model

Build the baseline model

In [ ]:
model = build_baseline_model()

model.summary()

Fit the baseline model

In [ ]:
history = model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_val, y_val))

Plot the Loss Curves

In [ ]:
plt.subplot(2, 1, 1)
plt.title('Loss')
plt.plot(history.history['loss'], color='blue', label='training loss')
plt.plot(history.history['val_loss'], color='orange', label='validation loss')
plt.legend(loc="upper right")

plt.subplots_adjust(hspace=0.5)
plt.figure(figsize=(3,4), dpi=300)

plt.show()

Score trained model and predictions.

In [ ]:
# Score trained model.
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

# make prediction.
pred = model.predict(X_test)

##Scale up##

Define and compile the CNN model

In [ ]:
# Define the CNN model

def build_model(dropouts_enabled = False, learning_rate = 0.001):
  
  model = Sequential()

  model.add(Conv2D(128, (3, 3), activation='relu', input_shape=(32, 32, 3)))
  model.add(MaxPooling2D((2, 2)))
  if (dropouts_enabled): model.add(Dropout(0.25))

  model.add(Conv2D(256, (3, 3), activation='relu'))
  model.add(MaxPooling2D((2, 2)))
  if (dropouts_enabled): model.add(Dropout(0.25))

  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dense(128, activation="relu"))
  if (dropouts_enabled): model.add(Dropout(0.5))
  model.add(Dense(10))

  # Compile the model
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])

  print(f"Learning Rate = {learning_rate}.  Dropouts are {'enabled' if dropouts_enabled else 'not enabled'}.\n")

  model.summary()

  return model

Create a callback that will stop fitting the model once it reaches 95% accuracy.

In [ ]:
class callback_val_accuracy(tf.keras.callbacks.Callback):
  def __init__(self, accuracy):
    self.accuracy = accuracy

  def on_epoch_end(self, epoch, logs=[]):
    if (logs.get("val_accuracy") >= self.accuracy):
      self.model.stop_training = True
      print(f"\nReached {self.accuracy*100} validation accuracy so cancelling training after {epoch} epochs.\n")

Build the model

In [ ]:
model=build_model(dropouts_enabled=False, learning_rate=0.0001)

Fit the CNN model

In [ ]:
# Train the model
callbacks = callback_val_accuracy(0.99)

#history = model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_val, y_val), callbacks=[callbacks])

history = model.fit(X_train, y_train, epochs=200, batch_size=128, validation_data=(X_val, y_val))

Loss and Accuracy

In [ ]:
plt.subplot(2, 1, 1)
plt.title('Loss')
plt.plot(history.history['loss'], color='blue', label='training loss')
plt.plot(history.history['val_loss'], color='orange', label='validation loss')
plt.legend(loc="upper right")

plt.subplot(2, 1, 2)
plt.title('Accuracy')
plt.plot(history.history['accuracy'], color='blue', label='training accuracy')
plt.plot(history.history['val_accuracy'], color='orange', label='validation accuracy')
plt.legend(loc="lower right")

plt.subplots_adjust(hspace=0.5)
plt.figure(figsize=(3,4), dpi=300)

plt.show()

Evaluate the model

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

# Convolutional Neural Network with Tuning

Install and import Tuner

In [ ]:
!pip install keras-tuner --upgrade
import keras_tuner
keras_tuner_version = keras_tuner.__version__
print(f"keras_tuner version: {keras_tuner_version}")

Create a tunable model

In [ ]:


def build_model(hp):
  model = Sequential()

  model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (3, 3), activation='relu'))
  model.add(MaxPooling2D((2, 2)))
  
  model.add(Flatten())
  model.add(Dense(
        # tune number of units
        units = hp.Int('units', min_value=32, max_value=512, step=32),
        # tune the activation function to use
        activation=hp.Choice("activation", ["relu", "tanh"])))
  model.add(Dense(10, activation="softmax"))
  
  learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])
  
  return model

Build and Compile the model

In [ ]:
build_model(keras_tuner.HyperParameters())

Create the tuner

In [ ]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective='val_accuracy',
    max_trials=100,
    executions_per_trial=2,
    directory='my_dir',
    project_name = "cifar10-image-classification-cnn")

Execute a search for the best model

In [ ]:
stop_early =  callback_val_accuracy(0.995)

tuner.search(X_train, y_train, epochs=100, validation_data=(X_val, y_val), callbacks=[stop_early])
best_model = tuner.get_best_models()[0]
print(f"Search Space Summary:\n{tuner.search_space_summary}")